In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params, gaussian_kernel
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import unet_builder as bare_model_builder

In [2]:
PROJECT_NAME = 'unet'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 30
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train_original, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train_original, y_val_original = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

In [4]:
y_base_train = np.array([gaussian_kernel(y) for y in y_base_train_original])
y_train = np.array([gaussian_kernel(y) for y in y_train_original])
y_val = np.array([gaussian_kernel(y) for y in y_val_original])

y_base_train.shape, y_train.shape, y_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [5]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_unet_depth = hp.Int("unet_depth", min_value=0, max_value=3, step=1, default=2)
    hp_n_conv_layers = hp.Int("n_conv_layers", min_value=1, max_value=3, step=1)
    hp_filters_mult = hp.Choice("conv_filters_mult", values=[1, 2, 4, 8, 16], default=4)
    hp_spatial_dropout = hp.Choice("conv_spatial_dropout", values=[0.0, 0.1, 0.2])
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_unet_depth, hp_n_conv_layers, hp_filters_mult, hp_spatial_dropout,
                               hp_batch_normalization, hp_input_batch_normalization, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [6]:
model_builder(kt.HyperParameters()).summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 reshape (Reshape)              (None, 24, 1)        0           ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 24, 32)       96          ['reshape[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 12, 32)       0           ['conv1d[0][0]']                 
                                                                                              

In [7]:
model_callbacks = [
    callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [8]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [9]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': False, 'normalize_signal': False}
Number of parameters: 367041
========== Model 1 ==========
{'unet_depth': 3, 'n_conv_layers': 1, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.1, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 16929
========== Model 2 ==========
{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.1, 'batch_normalization': True, 'input_batch_normalization': False, 'normalize_signal': True}
Number of parameters: 370625
========== Model 3 ==========
{'unet_depth': 2, 'n_conv_layers': 2, 'conv_filters_mult': 16, 'conv_spatial_dropout': 0.1, 'batch_normalization': True, 'input_batch_normalization': False, 'normalize_signal': True}
Number of parameters: 1401601
========== Model 4 ==========
{'u

# Cross-validation for top 5 models

In [10]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC,
                                           overwrite=OVERWRITE)
model_scores = cross_validator()

{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': False, 'normalize_signal': False}
Number of parameters: 367041
Got score: 13.2888 (13.3120, 13.2657)
Got score: 19.3059 (13.0917, 25.5201)
Got score: 18.9127 (25.0426, 12.7827)
Got score: 13.1037 (13.1016, 13.1058)
Got score: 13.5185 (13.6302, 13.4068)


{'unet_depth': 3, 'n_conv_layers': 1, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.1, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 16929
Got score: 13.3617 (13.3931, 13.3303)
Got score: 13.0995 (13.1337, 13.0654)
Got score: 12.9463 (12.9657, 12.9269)
Got score: 13.2075 (13.1556, 13.2594)
Got score: 13.6416 (13.6867, 13.5966)


{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.1, 'batch_normalization': True, 'input_batch_normalization': False, 'normalize_signal': True}
Number of parameters: 370625
Got score: 13.5072 (13.5801, 13.4342)
Got score: 13.2451 (13.1977, 13.2925)
Got score: 13.0513 (13.0515, 13.0510)
Got score: 13.2845 (13.3522, 13.2167)
Got score: 13.7860 (13.7136, 13.8585)


{'unet_depth': 2, 'n_conv_layers': 2, 'conv_filters_mult': 16, 'conv_spatial_dropout': 0.1, 'batch_normalization': True, 'input_batch_normalization': False, 'normalize_signal': True}
Number of parameters: 1401601
Got score: 13.6845 (13.5241, 13.8449)
Got score: 13.4812 (13.5017, 13.4607)
Got score: 13.1214 (13.1649, 13.0780)
Got score: 13.4113 (13.3917, 13.4310)
Got score: 13.8947 (13.8716, 13.9178)


{'unet_depth': 2, 'n_conv_layers': 1, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.1, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 62049
Got score: 13.6587 (13.5784, 13.7391)
Got score: 13.2581 (13.2763, 13.2400)
Got score: 12.9688 (12.9405, 12.9970)
Got score: 13.2743 (13.2307, 13.3179)
Got score: 13.7207 (13.8669, 13.5746)


In [11]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,15.63,2.85,367041
1,13.25,0.24,16929
2,13.37,0.25,370625
3,13.52,0.26,1401601
4,13.38,0.28,62049
